In [ ]:
%load_ext autoreload

In [4]:
%autoreload 2
import tensorflow as tf

from local_tabnet import TabNet as LocalTabNet
from google_tabnet import TabNet as GoogleTabNet

In [5]:
%autoreload 2
FEAT_DIM = 5
ATTN_DIM = 2
OUTPUT_DIM = 1
OUTPUT_ACTIVATION = None

tf.random.set_seed(1)
model = LocalTabNet(
    FEAT_DIM,
    ATTN_DIM,
    OUTPUT_DIM,
    OUTPUT_ACTIVATION,
    sparsity=1
)

2024-02-19 14:39:25.724400: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-02-19 14:39:25.724479: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (al3615): /proc/driver/nvidia/version does not exist
2024-02-19 14:39:25.725209: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
dummy_in = tf.ones((1, FEAT_DIM), dtype="float")
model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae", "mse"]
)
model.predict(dummy_in)

2024-02-19 14:39:31.279602: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


array([[-0.1268824]], dtype=float32)

In [7]:
model.summary()

Model: "tab_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared_feature_layer (Shared multiple                  152       
_________________________________________________________________
feat_0 (FeatureTransformer)  multiple                  296       
_________________________________________________________________
feat_1 (FeatureTransformer)  multiple                  296       
_________________________________________________________________
feat_2 (FeatureTransformer)  multiple                  296       
_________________________________________________________________
feat_3 (FeatureTransformer)  multiple                  296       
_________________________________________________________________
feat_4 (FeatureTransformer)  multiple                  296       
_________________________________________________________________
feat_5 (FeatureTransformer)  multiple                  296 

In [8]:
model.metrics_names

['mask_entropy']

# Run a step of fit and validate

In [7]:
dummy_dataset = tf.data.Dataset.from_tensor_slices(
    (
        tf.ones((1, FEAT_DIM), dtype="float"),
        tf.ones((1, OUTPUT_DIM), dtype="float")
    )
).batch(1)
model.fit(
    dummy_dataset,
    epochs=1,
    batch_size=1,
    verbose=1,
    validation_data=dummy_dataset,
    validation_freq=1,
)

model.metrics_names

1/1 [==============================] - 11s 11s/step - loss: 2.6094 - mae: 1.0000 - mse: 1.0000 - mask_entropy: 1.6094 - val_loss: 2.9104 - val_mae: 1.1533 - val_mse: 1.3302 - val_mask_entropy: 1.5803


['loss', 'mae', 'mse', 'mask_entropy']

In [9]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

small_model = Sequential([
    Dense(25, input_shape=(FEAT_DIM,), activation="relu"),
    Dense(25, activation="relu"),
    Dense(1, activation="relu")
])
small_model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae", "mse"]
)
small_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 25)                150       
_________________________________________________________________
dense_20 (Dense)             (None, 25)                650       
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 26        
Total params: 826
Trainable params: 826
Non-trainable params: 0
_________________________________________________________________


In [10]:
small_model.fit(
    dummy_dataset,
    epochs=1,
    batch_size=1,
    verbose=1,
    validation_data=dummy_dataset,
    validation_freq=1,
)

1/1 [==============================] - 0s 429ms/step - loss: 1.0000 - mae: 1.0000 - mse: 1.0000 - val_loss: 1.0000 - val_mae: 1.0000 - val_mse: 1.0000


In [11]:
11/429e-3

25.641025641025642

# Compare to google implementation
Currently not working

In [ ]:
%autoreload 2
tf.random.set_seed(1)
model = GoogleTabNet(
    columns=[tf.feature_column.numeric_column(str(ci)) for ci in range(FEAT_DIM)],
    num_features=FEAT_DIM,
    feature_dim=ATTN_DIM,
    output_dim=ATTN_DIM,
    num_decision_steps=5,
    relaxation_factor=1.5,
    batch_momentum=0.95,
    virtual_batch_size=None,
    num_classes=OUTPUT_DIM
)
decision, total_entropy = model.encoder(dummy_in, reuse=False, is_training=False)
out = model.regress(decision, reuse=False)